<a href="https://colab.research.google.com/github/ragabhumi/Pelatihan_Earth_Imaging/blob/main/Pemodelan_Gayaberat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pengolahan Data Gayaberat Global (Satelit)**
**Pelatihan Teknis Earth Imaging**

Bogor, 15 Juni 2023

Oleh: Fildzah Zainati Fadhilah & Yosi Setiawan

---



Pada bagian ini akan diuraikan pengolahan gayaberat menggunakan software  Fatiando a Terra (Uieda et.al., 2013) untuk analisis data gayaberat dan PyGMT (Uieda et.al., 2023) untuk visualisasi. Kedua software tersebut berbasis Python.



Data yang digunakan adalah data gayaberat global GGMPlus.
Data GGMPlus dapat diunduh [disini](https://murray-lab.caltech.edu/GGMplus/index.html) dengan memilih tipe data **gravity** (free air anomaly) dan memasukkan koordinat batas area kajian. Penjelasan setiap tipe data dapat dilihat pada laman [berikut](http://ddfe.curtin.edu.au/gravitymodels/GGMplus/GGMplus_readme.dat). Data topografi dapat diunduh pada link [berikut](https://bgi.obs-mip.fr/data-products/grids-and-models/modele-global-ggmplus2013/). Namun, pada modul ini, data GGMPlus yang akan diolah sudah disiapkan pada Github. Berikut dijelaskan tahap-tahap pengolahan data gayaberat:

## **Persiapan Perangkat Lunak Pengolahan**

---



1. Salin file Jupyter Notebook ini dengan klik tombol **Copy to Drive** agar dapat memodifikasi dan menyimpan perubahan-perubahan yang dilakukan. Pada setiap cell kode, terdapat tombol *Run cell* di sebelah kiri setiap cell. Ikuti perintah dengan klik tombol *Run cell* secara berurutan.

2. Install CondaColab di Google Colab di akun Google masing-masing. Pada setiap cell kode, terdapat tombol *Run cell* di sebelah kiri *cell*. Ikuti perintah dengan klik tombol *Run cell* secara berurutan. Tunggu hingga proses instalasi selesai. Jika muncul notifikasi *Your session crashed for an unknown reason*, abaikan dan tutup notifikasi tersebut.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

3. Cek apakah instalasi CondaColab berhasil. Jika berhasil maka akan muncul *Everything looks OK!*

In [ ]:
import condacolab
condacolab.check()

4. Instal modul-modul Python yang diperlukan. Tunggu hingga proses selesai.

In [ ]:
!mamba install verde harmonica boule pooch pygmt pyproj pandas xarray

5. Import modul-modul Python yang akan digunakan

In [ ]:
# The standard Python science stack
import numpy as np
import pandas as pd
import xarray as xr

# For projections (wrapped for Proj)
import pyproj

# Plotting maps using GMT
import pygmt

# The Fatiando stack
import pooch
import verde as vd
import boule as bl
import harmonica as hm

## **Persiapan Data**

---



1. Data gayaberat dan topografi dari GGMPlus yang digunakan dalam modul ini telah diupload di Github. Untuk mengunduhnya kita gunakan modul Pooch.

In [ ]:
url_grav = "https://github.com/ragabhumi/Pelatihan_Earth_Imaging/raw/main/data_gravity/GGMPlus_Gravity_96_97_4_5.csv"
md5_grav = "md5:cc3238dec6eb3f60bb017eabf66ae1a3"
path_grav = pooch.retrieve(url_grav, known_hash=md5_grav)
print(path_grav)

url_topo = "https://github.com/ragabhumi/Pelatihan_Earth_Imaging/raw/main/data_gravity/ERTM_96_97_4_5.grd"
md5_topo = "md5:ac76092d84f68ce102cc055061ebcbdf"
path_topo = pooch.retrieve(url_topo, known_hash=md5_topo)
print(path_topo)

2. Cek data gayaberat yang telah diimport dalam format txt/csv. Kolom FAA yang bernilai NaN menunjukkan bahwa pada lokasi tersebut adalah wilayah laut. Namun pada modul ini, nilai anomali NaN akan dihapus agar dapat diproses pada saat pembentukan trend pemisahan anomali regional-residual.

In [ ]:
data = pd.read_csv(path_grav,names=['longitude','latitude','FAA']).dropna()
data

3. Cek data topografi yang berformat netCDF.

In [ ]:
topography = xr.load_dataarray(path_topo)
topography

 4. Pada data satelit, tidak terdapat data elevasi pada setiap titik, sehingga perlu mengekstrak nilai ketinggian dari data grid topografi.

In [ ]:
data = pygmt.grdtrack(grid=topography, points=data, newcolname='elevation')
data

5. Plot data gayaberat dan topografi.

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="seis", series=[data.FAA.min(), data.FAA.max()])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.FAA,
    cmap=True,
    style="c4p",
    projection="M15c",
    frame=True,
)
fig.coast(resolution='f',shorelines='1p')
fig.colorbar(frame='af+l"FAA [mGal]"')
fig.show()

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="earth", series=[topography.values.min(), topography.values.max()])
fig.grdimage(topography, shading=True, projection="M15c", frame=True)
fig.coast(resolution='f',shorelines='1p')
fig.colorbar(frame='af+l"topography [m]"')
fig.show()

In [ ]:
topo_plain = vd.project_grid(topography, projection=projection)
topo_plain

## **Koreksi Bouguer**

---



1. Menghitung nilai koreksi Bouguer di setiap data titik. Nilai densitas yang digunakan adalah 2670 kg/m^3

In [ ]:
bouguer_correction = hm.bouguer_correction(data.elevation)
data["bouguer_corr"] = bouguer_correction
data


2. Menghitung nilai Anomali Bouguer Sederhana / Simple Bouguer Anomaly (SBA)

In [ ]:
data["bouguer_anomaly"] = data.FAA - bouguer_correction
data

3. Plot peta SBA.

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="seis", series=[data.bouguer_anomaly.min(), data.bouguer_anomaly.max()])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.bouguer_anomaly,
    cmap=True,
    style="c4p",
    projection="M15c",
    frame=True,
)
fig.coast(resolution='f',shorelines='1p')
fig.colorbar(frame='af+l"simple bouguer anomaly [mGal]"')
fig.show()

## **Pemisahan Anomali Regional-Residual**

---



1. Pemisahan anomali regional-residual pada latihan ini menggunakan metode polinomial 2D

In [ ]:
coordinates = (data.longitude, data.latitude)
trend = vd.Trend(degree=3).fit(coordinates, data.bouguer_anomaly)

data["regional_anomaly"] = trend.predict(coordinates)
data["residual_anomaly"] = data.bouguer_anomaly - trend.predict(coordinates)
data

2. Plot peta anomali regional.

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="seis", series=[data.regional_anomaly.min(), data.regional_anomaly.max()])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.regional_anomaly,
    cmap=True,
    style="c4p",
    projection="M15c",
    frame=True,
)
fig.coast(resolution='f',shorelines='1p')
fig.colorbar(frame='af+l"regional anomaly [mGal]"')
fig.show()

2. Plot peta anomali residual.

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="seis", series=[data.residual_anomaly.min(), data.residual_anomaly.max()])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.residual_anomaly,
    cmap=True,
    style="c4p",
    projection="M15c",
    frame=True,
)
fig.coast(resolution='f',shorelines='1p')
fig.colorbar(frame='af+l"residual anomaly [mGal]"')
fig.show()

### Daftar Pustaka

Uieda, L., V. C. Oliveira Jr, and V. C. F. Barbosa (2013), Modeling the Earth with Fatiando a Terra, Proceedings of the 12th Python in Science Conference, pp. 91-98. doi:10.25080/Majora-8b375195-010.



Uieda, L., Tian, D., Leong, W. J., Schlitzer, W., Grund, M., Jones, M., Fröhlich, Y., Toney, L., Yao, J., Magen, Y., Tong, JH., Materna, K., Belem, A., Newton, T., Anant, A., Ziebarth, M., Quinn, J., and Wessel, P. (2023). PyGMT: A Python interface for the Generic Mapping Tools (Version 0.9.0). doi:10.5281/zenodo.7772533.